In [1]:
 ! pip install -q kaggle

In [2]:
 from google.colab import files

In [ ]:
files.upload()

In [5]:
! mkdir ~/.kaggle

In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets download -d puneet6060/intel-image-classification

 96% 333M/346M [00:02<00:00, 118MB/s] 
100% 346M/346M [00:03<00:00, 114MB/s]


In [9]:
!sudo apt install unzip

Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-440
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
!unzip -q "intel-image-classification.zip"

In [11]:
!ls

 intel-image-classification.zip   kaggle.json   seg_pred   seg_train
'kaggle (1).json'		  sample_data   seg_test


In [12]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import BatchNormalization    
from keras.preprocessing.image import ImageDataGenerator
vgg19_model= keras.applications.vgg19.VGG19()
vgg19_model.summary()
model = Sequential()
i=0
for layer in vgg19_model.layers:
    if i > 24:
      break
    i+=1
    model.add(layer)
model.summary()
for layer in model.layers:
    layer.trainable = False
model.add(BatchNormalization())
model.add(Dense(6,activation = "softmax"))
model.summary()

574717952/574710816 [==============================] - 4s 0us/step
Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [14]:
model.compile(optimizer="adam",loss = "categorical_crossentropy",metrics=["accuracy"])
classifier=model
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set= train_datagen.flow_from_directory(
        './seg_train/seg_train',
        target_size=(224, 224),
        batch_size=64)

test_set = test_datagen.flow_from_directory(
        './seg_test/seg_test',
        target_size=(224, 224),
        batch_size=64)

classifier.fit_generator(
        training_set,
        steps_per_epoch=14034//64,
        epochs=100,
        validation_data=test_set,
        validation_steps=3000//64)

Found 14034 images belonging to 6 classes.
Found 3000 images belonging to 6 classes.
Epoch 1/100
219/219 [==============================] - 188s 858ms/step - loss: 0.5411 - accuracy: 0.8037 - val_loss: 0.5544 - val_accuracy: 0.8186
Epoch 2/100
219/219 [==============================] - 187s 854ms/step - loss: 0.4422 - accuracy: 0.8365 - val_loss: 0.4068 - val_accuracy: 0.8438
Epoch 3/100
219/219 [==============================] - 184s 841ms/step - loss: 0.4065 - accuracy: 0.8499 - val_loss: 0.3725 - val_accuracy: 0.8594
Epoch 4/100
219/219 [==============================] - 181s 829ms/step - loss: 0.3940 - accuracy: 0.8591 - val_loss: 0.3640 - val_accuracy: 0.8702
Epoch 5/100
219/219 [==============================] - 181s 827ms/step - loss: 0.3741 - accuracy: 0.8646 - val_loss: 0.3640 - val_accuracy: 0.8668
Epoch 6/100
219/219 [==============================] - 178s 815ms/step - loss: 0.3704 - accuracy: 0.8654 - val_loss: 0.3825 - val_accuracy: 0.8628
Epoch 7/100
219/219 [============